In [8]:
# Tutorial for the scraping:
# https://www.youtube.com/watch?v=8dTpNajxaH0

import pandas as pd
from bs4 import BeautifulSoup
import requests

In [11]:
import datetime

# Define the start and end dates
start_date = datetime.datetime(2024, 7, 12)
end_date = datetime.datetime(2024, 11, 2)

# Create a list to store the URLs
urls = []

# Loop through the date range
current_date = start_date
while current_date <= end_date:
    # Format the date as DD.MM.YYYY
    formatted_date = current_date.strftime('%d.%m.%Y')
    # Create the URL
    url = f'https://www.eso.bg/imb/?imbalance-prices&date={formatted_date}'
    # Append the URL to the list
    urls.append(url)
    # Increment the current date by one day
    current_date += datetime.timedelta(days=1)



In [12]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

# table with prices from ESO
tablePrices = soup.find_all('table')[0]
listTitles = tablePrices.find_all('th')
listTitles

#get titles of the dataframe from ESO
df_Prices = pd.DataFrame(columns= [titles.text.strip() for titles in listTitles])

for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    # table with prices from ESO
    tablePrices = soup.find_all('table')[0]
    listTitles = tablePrices.find_all('th')
    listTitles

    # find tr tag in table where prices are from ESO
    columnData  = tablePrices.find_all('tr')

    for row in columnData[1:]:
        row_data = row.find_all('td')
        individualRowData = [data.text.strip() for data in row_data]
        
        lenght = len(df_Prices)
        df_Prices.loc[lenght] = individualRowData


    # Strip to get the desired format "DD.MM.YYYY HH:MM"
    df_Prices['Сетълмент период'] = df_Prices['Сетълмент период'].str.split('-').str[0]
    df_Prices['Сетълмент период'] = df_Prices['Сетълмент период'].apply(lambda x: f"{x[:16]}")

In [13]:
df_Prices

,Сетълмент период,Прогнозна цена за недостиг (лв/MWh),Прогнозна ценаза излишък (лв/MWh)
0,12.07.2024 00:00,931.67,931.67
1,12.07.2024 00:15,561.10,561.10
2,12.07.2024 00:30,562.12,562.12
3,12.07.2024 00:45,936.19,936.19
4,12.07.2024 01:00,0.00,0.00
...,...,...,...
11052,02.11.2024 23:00,0.00,0.00
11053,02.11.2024 23:15,1.00,1.00
11054,02.11.2024 23:30,1.00,1.00
11055,02.11.2024 23:45,1.00,1.00


In [14]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

# table with imbalance from ESO
imbalanceTable = soup.find_all('table')[1]
listTitles = imbalanceTable.find_all('th')
listTitles

#get titles of the dataframe from ESO
df_imbalance = pd.DataFrame(columns= [titles.text.strip() for titles in listTitles])

for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    # table with prices from ESO
    imbalanceTable = soup.find_all('table')[1]
    listTitles = imbalanceTable.find_all('th')
    listTitles

    # find tr tag in table where prices are from ESO
    columnData  = imbalanceTable.find_all('tr')

    for row in columnData[1:]:
        row_data = row.find_all('td')
        individualRowData = [data.text.strip() for data in row_data]
        
        lenght = len(df_imbalance)
        df_imbalance.loc[lenght] = individualRowData


df_imbalance['Сетълмент период'] = df_imbalance['Сетълмент период'].str.split('-').str[0]
df_imbalance['Сетълмент период'] = df_imbalance['Сетълмент период'].apply(lambda x: f"{x[:16]}")
df_imbalance

,Сетълмент период,Прогнозен небаланс на системата (MWh)
0,12.07.2024 00:00,-0.496
1,12.07.2024 00:15,3.380
2,12.07.2024 00:30,5.671
3,12.07.2024 00:45,-2.774
4,12.07.2024 01:00,6.617
...,...,...
11052,02.11.2024 23:00,11.451
11053,02.11.2024 23:15,2.335
11054,02.11.2024 23:30,4.032
11055,02.11.2024 23:45,0.007
